# Introduction: 
In this challenge, we took a deep dive into the media service and original programming production company, Netflix. Investigating genre, reviews, and rating statistics to gain a better understanding of the company's strategy. 

# Main Hypothesis
If a title located on Netflix is classified under the "Action" genre, then it is more likely to have a higher IMDB user rating.

# Research Question Number 1:
Which genre holds the most Netflix titles?
- To answer this question, we will first dataset of Netflix titles, clean and group it. 

In [4]:
#import dependencies 
import os 
import csv 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import json
import requests